In [1]:
import ee
import geemap


In [3]:
ee.Authenticate()
ee.Initialize(project='ee-group1project')

Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [18]:
point = ee.Geometry.Point([-97.733330, 30.266666])


image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2016-01-01", "2016-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

vis_params = {"min": 0, "max": 30000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

In [5]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
image.get("CLOUD_COVER").getInfo()

0.02

In [16]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)
print(points.size().getInfo())
print(points.first().getInfo())

5000
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-97.20153473377643, 29.805104025092593]}, 'id': '0', 'properties': {'landcover': 81}}


In [7]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a SVM classifier with default parameters.
trained = ee.Classifier.libsvm().train(features=training, classProperty=label, inputProperties=bands)

# Print the first feature of the training data to verify
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 8568, 'SR_B2': 8878, 'SR_B3': 10068, 'SR_B4': 10196, 'SR_B5': 14103, 'SR_B6': 13740, 'SR_B7': 11386, 'landcover': 81}}


In [8]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=27282.0, center=[30.26666600000001, -97.73333000000001], controls=(WidgetControl(options=['position…

In [9]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=27282.0, center=[30.26666600000001, -97.73333000000001], controls=(WidgetControl(options=['position…

In [10]:
print("Change layer opacity:")

cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))
Map.add_legend(builtin_legend="NLCD")
Map

Change layer opacity:


Map(bottom=27282.0, center=[30.268556249047727, -97.72931638306417], controls=(WidgetControl(options=['positio…

### Accuracy Assessment


In [11]:
# Sample validation points from the image using the same bands and label
validation = image.select(bands).sampleRegions(
    collection=points,
    properties=[label],
    scale=30

)

In [12]:
# Classify the validation dataset using the trained SVM model
validated = validation.classify(trained)


In [13]:
# Generate the confusion matrix for the validation data
confusion_matrix = validated.errorMatrix(label, 'classification')


In [14]:
# Print the confusion matrix and derived metrics
print('Confusion Matrix:', confusion_matrix.getInfo())

print('Overall Accuracy:', confusion_matrix.accuracy().getInfo())
print('Kappa Coefficient:', confusion_matrix.kappa().getInfo())

Confusion Matrix: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,